In [1]:
# ---  Initial Setup ---
import numpy as np
import pandas as pd
import os
from pathlib import Path

print("Environment ready.")
print("Kaggle input directory:", os.listdir("/kaggle/input"))

Environment ready.
Kaggle input directory: ['agents-intensive-capstone-project']


# Study & Career Coach Agent 

This project is my capstone for the 5-Day AI Agents Intensive Course with Google.
It is an AI agent that helps college students plan study schedules, track tasks
and get quick guidance for exams and placements.

It uses:
- AI agents with Gemini + ADK
- Custom tools for study planning and resource summarization
- Session and long-term memory for personalization
- Logs and evaluation methods to monitor quality


## Problem Statement

College students often struggle to manage multiple exams, assignments and placement preparation.  
They need help planning study schedules,tracking tasks and staying consistent.

This agent acts as a STUDY & CAREER COACH that:
- Creates personalized study plans  
- Manages tasks and deadlines  
- Remembers user preferences  
- Helps students stay organized and exam-ready  


## Tools Used in This Agent
### 1.Study Plan Tool
This tool takes:
- List of subjects  
- Exam dates or number of days left  
- Hours available per day  

It generates a daily or weekly study schedule.

### 2.Task Manager Tool
This tool allows:
- Adding tasks with deadlines  
- Listing all tasks  
- Updating completed tasks  

These tools will be implemented as custom Python functions and integrated into the agent using ADK.


## Memory Plan

This agent will use two types of memory:

### 1.Session Memory
- Remembers the current conversation  
- Keeps track of what the user asked recently  
- Helps the agent provide connected multi-turn answers  

### 2. Long-Term Memory
The agent will store the following information permanently:
- User name  
- Branch and year (e.g CSE 4th year)  
- Subjects and upcoming exam dates  
- User's preferred study hours per day  
- Task list (pending and completed)

This allows the agent to give more personalized study plans and follow-up suggestions.


## Evaluation Plan

To ensure the quality of the agent,I will evaluate it in the following ways:

### 1. Logs and Traces
- Check which tools the agent calls
- Inspect the inputs and outputs of each tool
- Verify that the agent behaves as intended

### 2. Test Prompts
Run a set of prompts such as:
- "I have ML,OS and DBMS exams in 10 days.Create a study plan."
- "Add a task: Revise Unit 3 of OS by tomorrow."
- "List all my tasks."

I will check if:
- The study plan is realistic and balanced  
- The tasks are stored correctly  
- The responses are consistent across multiple turns

### 3. Human Feedback
I(user) will test the agent and give feedback to improve:
- Clarity  
- Relevance  
- Consistency  

This completes the evaluation approach for the capstone.


In [2]:
import sys, os
print("platform:", sys.platform)
print("kernel argv:", sys.argv[:3])
print("KAGGLE_INPUT exists:", os.path.exists("/kaggle/input"))
print("GOOGLE_API_KEY in env?", "GOOGLE_API_KEY" in os.environ)
print("First 8 chars GOOGLE_API_KEY:", os.environ.get("GOOGLE_API_KEY","")[:8])



platform: linux
kernel argv: ['/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py', '-f', '/tmp/tmpfc2h8omn.json']
KAGGLE_INPUT exists: True
GOOGLE_API_KEY in env? False
First 8 chars GOOGLE_API_KEY: 


In [3]:
# Load secret into environment variable manually
from kaggle_secrets import UserSecretsClient
import os

usc = UserSecretsClient()
key = usc.get_secret("GOOGLE_API_KEY")

os.environ["GOOGLE_API_KEY"] = key

print("Environment variable successfully loaded!")
print("First 6 chars:", os.environ["GOOGLE_API_KEY"][:6])


Environment variable successfully loaded!
First 6 chars: AIzaSy


In [4]:
!pip install -q google-generativeai

import os
import google.generativeai as genai

!pip install -q google-generativeai

import google.generativeai as genai
import os

api_key = os.environ.get("GOOGLE_API_KEY")
if not api_key:
    from kaggle_secrets import UserSecretsClient
    api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
try:
    genai.configure(api_key=api_key)
    print("✅ genai configured")
except Exception as e:
    print("genai.configure failed:", type(e).__name__, str(e))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 w

In [5]:
# --- Gemini Helper Function ---
import google.generativeai as genai

DEFAULT_MODEL = "models/gemini-2.5-flash"

def gemini_text(prompt, model_name=None):
    """
    Simple and stable wrapper to call Gemini.
    """
    model_to_use = model_name or DEFAULT_MODEL
    model = genai.GenerativeModel(model_to_use)
    resp = model.generate_content(prompt)
    return resp.text



In [6]:
# --- Personality System Instructions ---
def get_system_instruction(personality):
    personality = personality.lower().strip()

    if personality == "friendly senior":
        return (
            "You are a friendly senior student. "
            "You explain things clearly, supportively, and in a motivating tone. "
            "You give practical study advice and keep the response simple."
        )

    elif personality == "strict mentor":
        return (
            "You are a strict mentor. "
            "You speak directly, keep things disciplined, and focus on efficiency."
        )

    elif personality == "fun & energetic":
        return (
            "You are fun, energetic, and enthusiastic. "
            "You add excitement and make studying feel enjoyable."
        )

    # default personality
    return (
        "You are a helpful and clear AI assistant. "
        "Respond in a simple, structured, student-friendly way."
    )


In [7]:
def gemini_text_with_personality(prompt: str, personality: str = "Default", model_name: str = None):
    """
    Adds a personality system instruction before calling gemini_text.
    """
    system_instruction = get_system_instruction(personality)
    final_prompt = system_instruction + "\n\n" + prompt
    return gemini_text(final_prompt, model_name=model_name)


In [8]:
#  Study Plan Tool (with personality support) ---

def study_plan(subjects, days_left, hours_per_day, personality: str = "Default", model_name: str = None):
    """
    Generate a study plan using Gemini with optional personality/style.

    Parameters:
    - subjects: list of subject names (e.g. ["ML","OS"]) or a comma-separated string "ML, OS"
    - days_left: int number of days until the exam
    - hours_per_day: float/int available study hours per day
    - personality: a style string (e.g., "Default", "Friendly Senior", "Tamil + English (Tanglish)")
    - model_name: optional model override string (e.g., "models/gemini-2.5-pro")

    Returns:
    - str: AI-generated study plan (human-readable)
    """

    if isinstance(subjects, list):
        subjects_list = subjects
    elif isinstance(subjects, str):
        subjects_list = [s.strip() for s in subjects.split(",") if s.strip()]
    else:
        raise ValueError("subjects must be a list or a comma-separated string")

    if not subjects_list:
        return "No subjects provided."

    try:
        days_left = int(days_left)
        hours_per_day = float(hours_per_day)
    except Exception:
        return "Invalid days_left or hours_per_day. Provide numeric values."

    subjects_text = ", ".join(subjects_list)

    prompt = f"""
You are a Study & Career Coach Agent helping a student prepare for exams.

Subjects: {subjects_text}
Days left: {days_left}
Hours per day: {hours_per_day}

Please produce:
1) A short motivating summary (3-6 sentences) explaining the overall strategy.
2) A clear day-by-day schedule labeled "Day 1", "Day 2", ... with subjects and hours for each day.
3) A short distribution note that explains roughly how hours are split across subjects.
4) 3 concise tips to follow the plan effectively.

Keep the language simple, actionable and encouraging. If the user requests a personality (tone), follow that tone.
"""

    if "gemini_text_with_personality" in globals():
        return gemini_text_with_personality(prompt, personality=personality, model_name=model_name)
    else:
        if personality and personality != "Default":
            sys_inst = get_system_instruction(personality) if "get_system_instruction" in globals() else ""
            full_prompt = sys_inst + "\n\n" + prompt if sys_inst else prompt
            return gemini_text(full_prompt, model_name=model_name)
        return gemini_text(prompt, model_name=model_name)


In [9]:
print(study_plan("ML, OS, DBMS", 7, 4, personality="Friendly Senior"))





Hey there! Seven days and 4 hours a day – that's a solid amount of time if we use it smartly. The key here isn't just about how much you study, but *how* you study. We're going to create a focused plan that spreads your effort across ML, OS, and DBMS, making sure you hit all the important areas. This strategy will help you build momentum, tackle tougher concepts, and feel really confident by exam day. You've got this!

---

### **Your 7-Day Exam Prep Schedule**

Here's a breakdown of how you can structure your 4 hours each day:

*   **Day 1 (4 hours):**
    *   ML (2 hours) - Focus on core concepts
    *   OS (2 hours) - Process management & scheduling
*   **Day 2 (4 hours):**
    *   DBMS (2 hours) - SQL queries & Normalization
    *   ML (2 hours) - Supervised learning models
*   **Day 3 (4 hours):**
    *   OS (2 hours) - Memory management
    *   ML (2 hours) - Unsupervised learning & evaluation
*   **Day 4 (4 hours):**
    *   DBMS (2 hours) - Transactions & Concurrency
    *   OS

In [10]:
# ---  Personality helper (clean) ---
from typing import List

def get_system_instruction(personality: str) -> str:
    if personality == "Tamil + English (Tanglish)":
        return (
            "You are a friendly Indian college senior talking to a junior in a mix of Tamil and English (Tanglish). "
            "Use simple words, short sentences, and be encouraging. "
            "Style example: 'Seri, first calm down. Ipdi panlam...' "
        )
    elif personality == "Friendly Senior":
        return (
            "You are a caring senior student giving advice to a junior. "
            "You are friendly, supportive and explain things in simple, clear English."
        )
    elif personality == "Strict Lecturer":
        return (
            "You are a strict but fair college lecturer. "
            "You speak in direct, no-nonsense English and focus on discipline and clarity."
        )
    elif personality == "Motivational Coach":
        return (
            'You are a motivational coach. You mix practical study tips with inspiring lines like '
            '"You can do this" and "One step at a time."'
        )
    elif personality == "Funny Friend":
        return (
            "You are a funny college friend. You keep answers light, a bit humorous, but still helpful and clear."
        )
    else:
        return (
            "You are a friendly, practical Study & Career Coach for college students. "
            "You reply in clear English, with short, structured answers."
        )

def list_personalities() -> List[str]:
    return [
        "Default",
        "Tamil + English (Tanglish)",
        "Friendly Senior",
        "Strict Lecturer",
        "Motivational Coach",
        "Funny Friend"
    ]



In [11]:
# ---  study_agent_reply (uses gemini_text_with_personality) ---
MODEL_NAME = "models/gemini-2.5-flash"  # keep dev default; you can override in calls

def study_agent_reply(user_message: str, tone: str = "Friendly Senior", model_name: str = None) -> str:
    """
    Short, personality-aware agent reply wrapper.
    Uses gemini_text_with_personality (must be defined).
    """
    system_instruction = get_system_instruction(tone) + (
        " If the student mentions exams or subjects, suggest a clear, short study strategy. "
        "Avoid very long paragraphs and give crisp steps or a short day-wise plan if applicable."
    )
    full_prompt = f"{system_instruction}\n\nUser: {user_message}"
    model_name = model_name or MODEL_NAME
    return gemini_text_with_personality(full_prompt, personality=tone, model_name=model_name)



In [12]:
# --- Task Manager (must exist before task-related cells) ---

class TaskManager:
    def __init__(self):
        self.tasks = []
        self.counter = 1  # unique IDs

    def add_task(self, title, deadline="no-deadline"):
        task = {
            "id": self.counter,
            "title": title,
            "deadline": deadline,
            "completed": False
        }
        self.tasks.append(task)
        self.counter += 1
        return task

    def list_tasks(self):
        return self.tasks

    def complete_task(self, task_id):
        for t in self.tasks:
            if t["id"] == task_id:
                t["completed"] = True
                return t
        return None

tm = TaskManager()

def format_tasks(tasks):
    """
    Convert task list to simple readable string.
    """
    if not tasks:
        return "No tasks available."
    lines = []
    for i, t in enumerate(tasks, start=1):
        status = "✔️" if t["completed"] else "⬜"
        lines.append(f"{i}. {status} {t['title']} (deadline: {t['deadline']})")
    return "\n".join(lines)


In [13]:
# --- Simple Memory for long-term storage  ---

class SimpleMemory:
    def __init__(self):
        self.store = {}

    def get(self, key, default=None):
        return self.store.get(key, default)

    def set(self, key, value):
        self.store[key] = value

    def remove(self, key):
        if key in self.store:
            del self.store[key]

memory = SimpleMemory()


In [14]:
# ---  user profile and personalized advice via memory ---
if memory.get("user") is None:
    memory.set("user", {
        "name": "Varsha",
        "branch": "CSE",
        "year": "4th year",
        "default_study_hours_per_day": 4,
        "exams": [
            {"subject": "ML", "days_left": 5},
            {"subject": "OS", "days_left": 5},
            {"subject": "DBMS", "days_left": 5},
        ]
    })

def update_user_profile(name=None, branch=None, year=None, default_hours=None):
    profile = memory.get("user", {})
    if name is not None:
        profile["name"] = name
    if branch is not None:
        profile["branch"] = branch
    if year is not None:
        profile["year"] = year
    if default_hours is not None:
        profile["default_study_hours_per_day"] = default_hours
    memory.set("user", profile)
    return profile

def personalized_study_advice(user_message: str, tone: str = "Default"):
    profile = memory.get("user", {})
    profile_text = (
        f"Name: {profile.get('name')}, Branch: {profile.get('branch')}, Year: {profile.get('year')}, "
        f"Default hours/day: {profile.get('default_study_hours_per_day')}. Upcoming exams: {profile.get('exams')}"
    )
    system_instruction = "You are a friendly Study & Career Coach who uses the user's profile to personalize advice."
    full_prompt = f"""{system_instruction}

Student profile:
{profile_text}

Student message:
{user_message}
"""
    return gemini_text_with_personality(full_prompt, personality=tone)


In [15]:
# --- Simple logger so agent tools don't break ---

import datetime

event_log = [] 

def log_event(action, details=None):
    """
    Store events (task added, completed, plan generated, etc.)
    Useful for debugging.
    """
    entry = {
        "timestamp": str(datetime.datetime.now()),
        "action": action,
        "details": details
    }
    event_log.append(entry)
    return entry


In [16]:
# ---  handle_user_message dispatcher (clean) ---
def handle_user_message(user_message: str) -> str:
    """
    Very simple dispatcher: basic commands + fallback to study agent reply.
    Commands supported (low-friction):
      - "add task <title> [| deadline]"  (use '|' to separate optional deadline)
      - "list tasks" or "show tasks"
      - "complete task <index>"
      - other text -> forwarded to study_agent_reply (or agent_with_study_plan if it mentions 'plan' or 'study')
    """
    msg = user_message.strip()
    low = msg.lower()

    if low.startswith("add task"):
        parts = msg[8:].strip().split("|")
        title = parts[0].strip()
        deadline = parts[1].strip() if len(parts) > 1 else "no-deadline"
        new = tm.add_task(title, deadline)
        log_event("add_task", {"task": new})
        return f"Task added: {new['title']} (deadline: {new['deadline']})"

    if low in ("list tasks", "show tasks"):
        return format_tasks(tm.list_tasks())

    if low.startswith("complete task"):
        try:
            num = int(low.split("complete task")[1].strip())
        except Exception:
            return "Please specify a valid number. Example: complete task 2"
        tasks = tm.list_tasks()
        if num < 1 or num > len(tasks):
            return "Invalid task number."
        task_id = tasks[num - 1]["id"]
        updated = tm.complete_task(task_id)
        log_event("complete_task", {"task": updated})
        return f"Completed: {updated['title']}"

    if "plan" in low or "study" in low:
        profile = memory.get("user", {})
        subj_list = [e["subject"] for e in profile.get("exams", [])] if profile.get("exams") else ["ML","OS","DBMS"]
        days = min([e["days_left"] for e in profile.get("exams", [])]) if profile.get("exams") else 7
        plan_text, explanation = agent_with_study_plan(user_message, subj_list, days, profile.get("default_study_hours_per_day",4))
        return f"{plan_text}\n\n---\n{explanation}"

    return study_agent_reply(user_message, tone="Friendly Senior")

## Architecture Overview

This capstone project implements a **STUDY & CAREER COACH AGENT** for students.

### Main Components
- **LLM Brain (Gemini via google-genai)**  
  - Handles natural language understanding  
  - Gives guidance, motivation, and explains plans  

- **Study Plan Tool (`generate_study_plan`)**  
  - Python function that builds a day-wise plan based on:
    - subjects
    - days left
    - hours per day  

- **Task Manager Tool (`add_task`, `list_tasks`, `complete_task`)**  
  - Stores tasks in an in-memory list  
  - Allows adding, viewing, and completing tasks  

- **Dispatcher (`handle_user_message`)**  
  - Detects simple intents from the user message:
    - "add task ..." → Task Manager tool  
    - "list tasks" / "show tasks" → Task list  
    - "complete task N" → Mark task done  
    - Messages with "plan"/"study" → Study Plan Tool + Agent explanation  
    - All other messages → Normal agent reply  

- **Chat Wrapper (`chat_with_agent`)**  
  - Simple helper for running demo conversations.


## Mapping to 5-Day AI Agents Intensive Concepts

- **Day 1 – Introduction to Agents**
  - This project defines a clear agent: a Study & Career Coach for students.
  - The agent takes user messages and decides actions using simple routing logic.

- **Day 2 – Tools & Interoperability**
  - The agent uses custom Python tools:
    - Study Plan Tool for building schedules  
    - Task Manager Tool for managing to-dos  

- **Day 3 – Context & Memory**
  - The task list acts as a simple form of memory within the session.
  - The agent can:
    - Remember previously added tasks  
    - Update and show them later  

- **Day 4 – Agent Quality**
  - Basic evaluation is done by:
    - Observing tool calls in the demo  
    - Testing different prompts (study plan, tasks, motivation)
    - Checking if outputs are realistic and useful  

- **Day 5 – Prototype to Production (Conceptual)**
  - This notebook acts as a prototype.
  - The same logic can be wrapped into:
    - A web API
    - A chatbot UI for students  
  - The architecture is simple and ready to be extended.


## How to Run This Notebook

1. **Install dependencies and configure API key**
   - The cell with `!pip install -q -U google-genai` and `client = genai.Client(api_key="...")` must be run first.

2. **Run tool and agent definitions**
   - Run the cells defining:
     - `generate_study_plan`, `print_study_plan`
     - `add_task`, `list_tasks`, `complete_task`
     - `study_agent_reply`
     - `agent_with_study_plan`
     - `handle_user_message`
     - `chat_with_agent`

3. **Run the final demo**
   - Execute the final demo cell to see:
     - Study plan generation
     - Task creation and completion
     - Motivational response from the agent

4. **Try your own inputs**
   - You can call:
     - `chat_with_agent("add task Revise ML unit 2 by tomorrow")`
     - `chat_with_agent("list tasks")`
     - `chat_with_agent("I have only 3 days for OS. Help me.")`


#  Study & Productivity AI Agent – Capstone Project  
### Kaggle x Google 5-Day AI Agents Intensive

This project implements a **Study & Productivity Assistant** AI Agent using:

- Gemini (via google-genai SDK)  
- Custom Tools (Study Plan Generator + Task Manager)  
- Rule-based Agent Dispatcher  
- Multi-step reasoning  
- Memory (task list across calls)  

### Agent Capabilities
- Generates personalized study plans  
- Adds tasks  
- Lists and completes tasks  
- Responds with motivational/academic guidance  
- Explains study plans generated by tools  

### Tools Implemented
- `generate_study_plan()` → Creates day-wise schedules  
- `add_task()` / `list_tasks()` / `complete_task()` → Productivity memory  
- `handle_user_message()` → Decides which tool to use  
- `chat_with_agent()` → Chat interface for demo  

###  Demo
The final demo block shows:
- Study plan generation  
- Task creation  
- Task completion  
- Natural conversation with the agent  

###  Mapping to Day 1–5 Concepts
- Day 1 → Agent structure  
- Day 2 → Tools  
- Day 3 → Memory  
- Day 4 → Evaluation (manual tests)  
- Day 5 → Prototype preparation for deployment  

This notebook demonstrates a real multi-tool agent that solves a real-world student productivity problem.


In [17]:
!pip install -q fpdf matplotlib


  Preparing metadata (setup.py) ... done


In [18]:
from fpdf import FPDF
from fpdf import FPDF
import matplotlib.pyplot as plt
from pathlib import Path

_last_plan = None        
_last_plan_text = ""

def update_last_plan(subjects, days_left, hours_per_day, tone: str = "Default", model_name: str = None):
    """
    Generate and store the latest plan and explanation for later export/analytics.
    Uses your study_plan() and agent_with_study_plan() functions.
    """
    global _last_plan, _last_plan_text

    plan_text = study_plan(subjects, days_left, hours_per_day, personality=tone, model_name=model_name)
    _last_plan_text = plan_text
    _last_plan = None
    return plan_text

def get_analytics_plot():
    """
    Create a matplotlib figure summarizing hours per subject based on _last_plan.
    If structured plan not available, returns None.
    """
    if not _last_plan:
        return None

    totals = {}
    for day_info in _last_plan:
        for block in day_info.get("study_blocks", []):
            subj = block.get("subject")
            hrs = float(block.get("hours", 0))
            totals[subj] = totals.get(subj, 0) + hrs

    fig, ax = plt.subplots()
    subjects = list(totals.keys())
    hours = list(totals.values())
    ax.bar(subjects, hours)
    ax.set_xlabel("Subject")
    ax.set_ylabel("Total Hours")
    ax.set_title("Study Hours per Subject")
    fig.tight_layout()
    return fig

def export_plan_pdf(filename: str = "/kaggle/working/study_plan.pdf"):
    """
    Export the last generated plan text to a PDF. Returns file path or None.
    """
    if not _last_plan_text:
        return None
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)
    for line in _last_plan_text.split("\n"):
        pdf.multi_cell(0, 8, line)
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    pdf.output(filename)
    return filename
import matplotlib.pyplot as plt

last_plan = None          
last_plan_text = ""       


def update_last_plan(subjects, days_left, hours_per_day, tone: str = "English"):
    """
    Generate a study plan + explanation and store it in global variables
    so we can use it for analytics and PDF export.
    """
    global last_plan, last_plan_text

    plan = generate_study_plan(subjects, days_left, hours_per_day)
    last_plan = plan
    last_plan_text = plan_to_text(plan)

    _, explanation = agent_with_study_plan(
        user_message="Generate a study plan for me.",
        subjects=subjects,
        days_left=days_left,
        hours_per_day=hours_per_day,
        tone=tone
    )
    return explanation


def get_analytics_plot():
    """
    Create a bar chart of total hours per subject from last_plan.
    Returns a matplotlib figure.
    """
    if not last_plan:
        return None

    totals = {}
    for day_info in last_plan:
        for block in day_info["study_blocks"]:
            subj = block["subject"]
            hrs = block["hours"]
            totals[subj] = totals.get(subj, 0) + hrs

    fig, ax = plt.subplots()
    subjects = list(totals.keys())
    hours = list(totals.values())

    ax.bar(subjects, hours)
    ax.set_xlabel("Subject")
    ax.set_ylabel("Total Hours (all days)")
    ax.set_title("Study Hours per Subject")

    return fig


def export_plan_pdf():
    """
    Export last_plan_text into a PDF file and return the file path.
    """
    if not last_plan_text:
        return None

    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    for line in last_plan_text.split("\n"):
        pdf.multi_cell(0, 8, line)

    file_path = "/kaggle/working/study_plan.pdf"
    pdf.output(file_path)
    return file_path


In [19]:
def recommend_resources_tool(subject: str, level: str = "intermediate") -> str:
    """
    Simple 'smart recommendation' tool.
    It does not call the LLM; it behaves like a deterministic tool,
    which the agent can then explain in its own personality.
    """

    subject = subject.upper().strip()

    base_lines = [f"Study recommendations for {subject} ({level} level):"]

    if subject == "ML":
        base_lines += [
            "- Focus topics: Supervised vs Unsupervised, Regression, Classification, SVM, Decision Trees, Overfitting/Regularization.",
            "- Practice: implement small models in Python (sklearn), and solve previous exam questions.",
            "- Resources: search on YouTube for 'Machine Learning crash course', 'ML exam revision playlist', and 'sklearn tutorial'."
        ]
    elif subject == "OS":
        base_lines += [
            "- Focus topics: Process vs Thread, Scheduling algorithms, Deadlocks, Memory management (paging/segmentation), File systems.",
            "- Practice: draw diagrams for process states, and manually solve scheduling problems (FCFS, SJF, RR, Priority).",
            "- Resources: search for 'Operating System exam prep', 'OS scheduling problems', 'deadlock explanation'."
        ]
    elif subject == "DBMS":
        base_lines += [
            "- Focus topics: SQL queries, Joins, Normalization, ER diagrams, Transactions and ACID properties.",
            "- Practice: write SQL queries for joins, aggregation, subqueries; draw ER diagrams for sample case studies.",
            "- Resources: search for 'DBMS SQL playlist', 'Normalization explained', 'ACID properties DBMS'."
        ]
    elif subject == "CN":
        base_lines += [
            "- Focus topics: OSI layers, TCP/IP, IP addressing, Routing, Congestion control.",
            "- Practice: solve numerical problems on subnetting and go through past question papers.",
            "- Resources: search 'Computer Networks exam revision', 'OSI 7-layer model', 'subnetting problems'."
        ]
    elif subject == "AI":
        base_lines += [
            "- Focus topics: Search algorithms, Knowledge representation, Inference, Basic ML, Agents.",
            "- Practice: write pseudo-code for algorithms like A*, DFS, BFS and solve example problems.",
            "- Resources: search 'AI agents concepts', 'search algorithms AI', 'AI exam revision'."
        ]
    else:
        base_lines += [
            "- Focus on core concepts, previous year question patterns, and summary notes.",
            "- Mix theory reading with problem solving.",
            "- Look for short crash-course videos and handwritten note summaries."
        ]

    return "\n".join(base_lines)


In [20]:
def handle_user_message_config(user_message: str,
                               tone: str = "Default",
                               selected_subjects=None,
                               days_left: int = None,
                               hours_per_day: int = None) -> str:
    """
    Dispatcher used by Gradio. Uses:
     - tm (TaskManager instance)
     - memory (SimpleMemory)
     - study_plan / agent_with_study_plan
     - recommend_resources_tool
     - revision_notes_tool
    """
    msg = (user_message or "").strip()
    low = msg.lower()
    subjects = selected_subjects or [e["subject"] for e in memory.get("user", {}).get("exams", [])] or ["ML", "OS", "DBMS"]
    days_left = int(days_left) if days_left is not None else memory.get("user", {}).get("exams", [{}])[0].get("days_left", 7)
    hours_per_day = int(hours_per_day) if hours_per_day is not None else memory.get("user", {}).get("default_study_hours_per_day", 4)

    if low.startswith("add task"):
        body = user_message[8:].strip()
        parts = body.split("|")
        title = parts[0].strip()
        deadline = parts[1].strip() if len(parts) > 1 else "no-deadline"
        new = tm.add_task(title, deadline)
        log_event("add_task", {"task": new})
        return f"Task added: {new['title']} (deadline: {new['deadline']})"

    if low in ("list tasks", "show tasks"):
        tasks = tm.list_tasks()
        return format_tasks(tasks)

    if low.startswith("complete task"):
        try:
            num = int(low.split("complete task", 1)[1].strip())
        except Exception:
            return "Please specify a valid number. Example: complete task 2"
        tasks = tm.list_tasks()
        if num < 1 or num > len(tasks):
            return "Invalid task number."
        task_id = tasks[num - 1]["id"]
        updated = tm.complete_task(task_id)
        log_event("complete_task", {"task": updated})
        return f"Completed: {updated['title']}"

    if any(k in low for k in ("recommend", "resources", "video", "playlist")):
        subj = subjects[0] if subjects else "general"
        base_recs = recommend_resources_tool(subj)
        prompt = f"Explain these recommendations in a student-friendly way:\n\n{base_recs}"
        return study_agent_reply(prompt, tone=tone)

    if any(k in low for k in ("revise", "revision", "notes")):
        topic = user_message
        return revision_notes_tool(topic)

    if "plan" in low or "study" in low:
        _, explanation = agent_with_study_plan(
            user_message=user_message,
            subjects=subjects,
            days_left=days_left,
            hours_per_day=hours_per_day,
            tone=tone
        )
        update_last_plan(subjects, days_left, hours_per_day, tone=tone)
        return explanation

    return study_agent_reply(user_message, tone=tone)




In [21]:
def revision_notes_tool(topic: str) -> str:
    """
    Generates crisp revision notes for any given topic.
    Notes follow a strict pattern:
    - Definition
    - Key points
    - Formula (if any)
    - Diagram idea
    - Exam pointers
    """

    topic_lower = topic.lower()

    notes = [f"📘 *Revision Notes for {topic}:*"]

    # OS Topics
    if "deadlock" in topic_lower:
        notes += [
            "• **Definition:** A state in multiprocessing where processes wait forever for resources held by each other.",
            "• **Conditions:** Mutual exclusion, Hold & Wait, No pre-emption, Circular wait.",
            "• **Prevention:** Break one of the 4 conditions.",
            "• **Avoidance:** Use Banker's Algorithm.",
            "• **Detection:** Resource Allocation Graph (RAG).",
            "• **Recovery:** Terminate processes or preempt resources.",
            "• **Diagram idea:** Draw a circular wait graph showing P1 → P2 → P3 → P1.",
            "• **Exam tip:** Always write the 4 conditions. 2 marks guaranteed."
        ]
    elif "scheduling" in topic_lower:
        notes += [
            "• **Definition:** CPU scheduling determines which process runs next on the CPU.",
            "• **Algorithms:** FCFS, SJF, SRTF, RR, Priority Scheduling.",
            "• **Key Terms:** Burst time, Waiting time, Turnaround time.",
            "• **Gantt Chart:** Mandatory for exam numericals.",
            "• **Exam tip:** Write formulas + solve one sample Gantt chart."
        ]

    # ML Topics
    elif "logistic" in topic_lower:
        notes += [
            "• **Definition:** Logistic Regression is used for binary classification.",
            "• **Activation:** Sigmoid function → 1/(1+e^-z)",
            "• **Cost Function:** Binary cross-entropy.",
            "• **Decision boundary:** Linear.",
            "• **When to use:** Spam detection, fraud detection.",
            "• **Exam tip:** Draw sigmoid curve + write cost formula."
        ]
    elif "svm" in topic_lower or "support vector" in topic_lower:
        notes += [
            "• **Definition:** SVM finds the best hyperplane separating classes with maximum margin.",
            "• **Key terms:** Margin, Support vectors, Hyperplane.",
            "• **Kernels:** Linear, RBF, Polynomial.",
            "• **Strength:** Works well with high dimensional data.",
            "• **Diagram:** Two classes with margin + support vectors."
        ]

    # DBMS Topics
    elif "normalization" in topic_lower:
        notes += [
            "• **Definition:** Process of reducing redundancy and improving data integrity.",
            "• **Forms:** 1NF → No repeating groups, 2NF → No partial dependency, 3NF → No transitive dependency.",
            "• **Diagram:** Table illustrating partial → full → transitive dependencies.",
            "• **Exam tip:** Always define partial & transitive dependency."
        ]
    elif "acid" in topic_lower:
        notes += [
            "• **Definition:** Set of properties ensuring reliable DB transactions.",
            "• **Atomicity:** All or nothing.",
            "• **Consistency:** Move DB from one valid state to another.",
            "• **Isolation:** Transactions do not interfere.",
            "• **Durability:** Changes are permanent.",
            "• **Exam tip:** Expand all four clearly for full marks."
        ]

    else:
        notes += [
            "• Definition (short)",
            "• Key points (3–5 bullets)",
            "• Simple formula (if any)",
            "• Diagram idea",
            "• Exam scoring tips"
        ]

    return "\n".join(notes)


In [22]:
def handle_voice_message(audio_path, tone, selected_subjects, days_left, hours_per_day, current_plan):
    """
    Prototype voice handler for Kaggle:
    - Receives recorded audio file path from Gradio
    - For this environment, we do NOT transcribe audio
      (full STT needs extra services and is better done outside Kaggle).
    - Instead, we show a clear message and keep the app stable.
    """

    if audio_path is None:

        helper_text = "🎤 No voice detected. Please record your question or type it in the textbox."
        return helper_text, current_plan

    helper_text = (
        "🎤 Voice message received.\n\n"
        "In this Kaggle prototype, full speech-to-text is not enabled.\n"
        "Please type your question in the textbox so I can answer in detail.\n\n"
        "This shows how a future voice mode would be integrated into the agent."
    )

    return helper_text, current_plan


In [23]:
import gradio as gr

def format_tasks_for_panel():
    tasks_list = tm.list_tasks() if "tm" in globals() else []
    if not tasks_list:
        return "No tasks added yet.\n\nTry commands like:\n- add task Revise OS unit 3 by tomorrow\n- list tasks"
    lines = []
    for idx, task in enumerate(tasks_list, start=1):
        status = "✅" if task.get("completed") else "⬜"
        deadline_text = f" (deadline: {task.get('deadline')})" if task.get("deadline") else ""
        lines.append(f"{idx}. {status} {task.get('title')}{deadline_text}")
    return "\n".join(lines)

def gradio_agent_chat(history, message, tone, selected_subjects, days_left, hours_per_day, current_plan):
    """
    Chat handler: uses config + keeps history + keeps plan box.
    Uses OpenAI-style message objects when chatbot.type == 'messages'.
    history: list of {"role": "user"/"assistant", "content": "..."}
    """
    if not message:
        return history or [], "", format_tasks_for_panel(), current_plan

    reply = handle_user_message_config(
        user_message=message,
        tone=tone,
        selected_subjects=selected_subjects,
        days_left=int(days_left),
        hours_per_day=int(hours_per_day)
    )

    history = history or []

    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": reply})

    return history, "", format_tasks_for_panel(), current_plan


def gradio_generate_plan(tone, selected_subjects, days_left, hours_per_day):
    """
    Generates and stores a study plan + explanation using the selected config.
    """
    subjects = selected_subjects or ["ML", "OS", "DBMS"]
    explanation = update_last_plan(
        subjects=subjects,
        days_left=int(days_left),
        hours_per_day=int(hours_per_day),
        tone=tone
    )
    return explanation

def clear_chat_only(tone, selected_subjects, days_left, hours_per_day, current_plan):
    """
    Clear only chat, keep tasks + plan.
    """
    return [], "", format_tasks_for_panel(), current_plan

def reset_all():
    """
    Clear chat + clear tasks + clear plan.
    """
    global _last_plan, _last_plan_text, tm
    if "tm" in globals():
        try:
            TaskManager  
            tm = TaskManager()
        except Exception:
            try:
                tm.tasks.clear()
            except Exception:
                pass
    _last_plan = None
    _last_plan_text = ""
    return [], "", format_tasks_for_panel(), ""

def generate_analytics_plot():
    fig = get_analytics_plot()
    return fig

def generate_plan_pdf_file():
    path = export_plan_pdf()
    return path

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 📚 Study & Productivity AI Agent")
    gr.Markdown(
        "A **study coach + task manager** agent.\n\n"
        "Try things like:\n"
        "- `Help me plan for my ML exam`\n"
        "- `add task Revise OS unit 3 by tomorrow`\n"
        "- `list tasks`\n"
        "- `complete task 1`"
    )

    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("### ⚙️ Settings")

            ui_mode = gr.Radio(
                ["Day / Light Focus", "Night / Dark Focus"],
                value="Day / Light Focus",
                label="UI Mode (concept)",
                info="Light vs night focus mood for your study plan wording."
            )

            tone_radio = gr.Radio(
                [
                    "Friendly Senior",
                    "Strict Lecturer",
                    "Motivational Coach",
                    "Funny Friend",
                    "Tamil + English (Tanglish)"
                ],
                value="Friendly Senior",
                label="Agent Personality"
            )

            subject_box = gr.CheckboxGroup(
                ["ML", "OS", "DBMS", "CN", "AI"],
                value=["ML", "OS", "DBMS"],
                label="Subjects to include in study plan"
            )
            days_slider = gr.Slider(
                minimum=1,
                maximum=30,
                value=5,
                step=1,
                label="Days left for exams"
            )
            hours_slider = gr.Slider(
                minimum=1,
                maximum=8,
                value=4,
                step=1,
                label="Study hours per day"
            )

    with gr.Row():

        with gr.Column(scale=3):
            gr.Markdown("### 💬 Chat with the Agent")

            chatbot = gr.Chatbot(
                label="Conversation",
                height=350,
                type="messages"
            )

            msg = gr.Textbox(
                label="Type your message",
                placeholder="Ask in English or Tanglish about exams, tasks, or motivation...",
            )
            voice_input = gr.Audio(
                type="filepath",
                label="🎤 Speak your question (record manually)"
            )

            voice_btn = gr.Button("Send Voice Message")

            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                clear_chat_btn = gr.Button("Clear Chat Only")
                reset_memory_btn = gr.Button("Reset Agent Memory (chat + tasks)")

        with gr.Column(scale=3):
            gr.Markdown("### 🗓️ Study Plan & Analytics")

            plan_box = gr.Textbox(
                value="Click 'Generate Study Plan' below to create a plan using the selected subjects/days/hours.",
                lines=12,
                interactive=False,
                label="Generated Study Plan"
            )
            with gr.Row():
                generate_btn = gr.Button("Generate Study Plan", variant="secondary")
                pdf_btn = gr.Button("Download Plan as PDF")
            analytics_plot = gr.Plot(label="Study Hours per Subject (Analytics)")

        with gr.Column(scale=2):
            gr.Markdown("### ✅ Your Tasks")
            tasks_box = gr.Textbox(
                value=format_tasks_for_panel(),
                lines=20,
                interactive=False,
                label=""
            )

    # Chat wiring
    msg.submit(
        fn=gradio_agent_chat,
        inputs=[chatbot, msg, tone_radio, subject_box, days_slider, hours_slider, plan_box],
        outputs=[chatbot, msg, tasks_box, plan_box]
    )
    send_btn.click(
        fn=gradio_agent_chat,
        inputs=[chatbot, msg, tone_radio, subject_box, days_slider, hours_slider, plan_box],
        outputs=[chatbot, msg, tasks_box, plan_box]
    )

    clear_chat_btn.click(
        fn=clear_chat_only,
        inputs=[tone_radio, subject_box, days_slider, hours_slider, plan_box],
        outputs=[chatbot, msg, tasks_box, plan_box]
    )

    reset_memory_btn.click(
        fn=reset_all,
        inputs=None,
        outputs=[chatbot, msg, tasks_box, plan_box]
    )

    generate_btn.click(
        fn=gradio_generate_plan,
        inputs=[tone_radio, subject_box, days_slider, hours_slider],
        outputs=[plan_box]
    )

    generate_btn.click(
        fn=generate_analytics_plot,
        inputs=None,
        outputs=[analytics_plot]
    )

    pdf_file_output = gr.File(label="Download your study_plan.pdf")

    pdf_btn.click(
        fn=generate_plan_pdf_file,
        inputs=None,
        outputs=[pdf_file_output]
    )

    voice_btn.click(
        fn=handle_voice_message,
        inputs=[voice_input, tone_radio, subject_box, days_slider, hours_slider, plan_box],
        outputs=[msg, plan_box]
    )


demo.launch(share=True)



* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://4279c99a6571189f6e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
